In [7]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [8]:
import pandas as pd
import numpy as np

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import json
import networkx as nx
import src.build_graph as bg

In [19]:
import logging
logger = logging.getLogger('debug')
hdlr = logging.FileHandler('/home/zjliao/dsc180a/debug.log', 'w')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.DEBUG)

In [20]:
logger.debug('testing')

In [21]:
ENV = json.load(open('/home/zjliao/dsc180a/config/env.json'))

In [22]:
cfg = json.load(open('/home/zjliao/dsc180a/config/test-params.json'))
logger.info(cfg)
malware = bg.malware_detection(**cfg)
A, apis, apks = malware.buildA_matrix()

get info
check malware
5550
Build A
extract smali
build adjacency matrix


In [24]:
apis

0                             java/lang/Object;-><init>()V
1        com/trid/tridad/TriDContentsView$Renderer;->is...
2        com/trid/tridad/TriDBrowser;->sendTouchMessage...
3        com/trid/tridad/TriDContentsView;->access$8()L...
4        android/util/Log;->w(Ljava/lang/String;Ljava/l...
                               ...                        
56074    com/soft/android/appinstaller/GlobalConfig;->g...
56075    com/soft/android/appinstaller/FirstActivity;->...
56076    com/soft/android/appinstaller/MemberActivity;-...
56077    com/soft/android/appinstaller/MemberActivity;-...
56078    com/soft/android/appinstaller/MemberActivity;-...
Length: 56079, dtype: object

,0
0,"(0, 0)\t1\n (0, 1)\t1\n (0, 2)\t1\n (0, 3..."
1,"(0, 0)\t1\n (0, 4)\t1\n (0, 12)\t1\n (0, ..."
2,"(0, 0)\t1\n (0, 4)\t1\n (0, 6)\t1\n (0, 1..."
3,"(0, 0)\t1\n (0, 4)\t1\n (0, 12)\t1\n (0, ..."
4,"(0, 0)\t1"
5,"(0, 0)\t1\n (0, 16)\t1\n (0, 18)\t1\n (0,..."
6,"(0, 0)\t1\n (0, 18)\t1\n (0, 24)\t1\n (0,..."
7,"(0, 0)\t1\n (0, 16)\t1\n (0, 18)\t1\n (0,..."
8,"(0, 0)\t1\n (0, 16)\t1\n (0, 18)\t1\n (0,..."
9,"(0, 0)\t1\n (0, 16)\t1\n (0, 18)\t1\n (0,..."


In [26]:
from scipy import sparse

In [29]:
A = sparse.csr_matrix(A)
A

<10x56079 sparse matrix of type '<class 'numpy.int64'>'
	with 73457 stored elements in Compressed Sparse Row format>

In [31]:
B_graph = malware.build_B()

Build B
create B graph


In [34]:
B = nx.adjacency_matrix(B_graph, apis)

In [35]:
B

<56079x56079 sparse matrix of type '<class 'numpy.int64'>'
	with 1075107 stored elements in Compressed Sparse Row format>

In [ ]:
P = nx.adjacency_matrix(malware.build_P(), apis)

In [ ]:
P

In [ ]:
A*A.T

In [ ]:
A*B*A.T

In [ ]:
A*B*P*P.T*B.T*A.T

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
x = A
y = [1 if(num<5) else 0 for num in range(10)]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
def AA(a):
    return a * a.T

def ABA(a, b):
    return a * b * a.T

def APA(a, p):
    return a * p * a.T

def APBPA(a, b, p):
    return a * p * b * p.T * a.T

In [ ]:
clf = SVC(kernel = 'precomputed')
kernel_train = AA(X_train)
clf.fit(kernel_train, y_train)
kernel_test = np.dot(X_test, X_train.T)
aa = clf.score(kernel_test, y_test)

In [ ]:
# ABA
clf = SVC(kernel = 'precomputed')
kernel_train = ABA(X_train, B)
clf.fit(kernel_train, y_train)
kernel_test = X_test * B * X_train.T
aba = clf.score(kernel_test, y_test)

In [ ]:
# APA
clf = SVC(kernel = 'precomputed')
kernel_train = X_train * P * X_train.T
clf.fit(kernel_train, y_train)
kernel_test = X_test * P * X_train.T
apa = clf.score(kernel_test, y_test)

In [ ]:
# APBPA
clf = SVC(kernel = 'precomputed')
kernel_train = APBPA(X_train, B, P)
clf.fit(kernel_train, y_train)
kernel_test = X_test * P * B * P.T * X_train.T
apbpa = clf.score(kernel_test, y_test)

In [ ]:
results = {
    'aa': aa,
    'aba': aba,
    'apa': apa,
    'apbpa': apbpa
}

In [ ]:
_, outpath = ENV

In [ ]:
outpath

In [ ]:
res = json.dumps(results)
outpath = ENV['output-paths']
f = open("%s/results.json"%outpath,"w")
f.write(res)
f.close()